In [1]:
import simpy
import numpy as np
import random
import pandas as pd
import networkx as nx

# Constants and Simulation Parameters
KB_TO_MB = 1 / 1024
BIT_TO_MBIT = 1 / 1024
GHZ_TO_HZ = 1e9
MHZ_TO_HZ = 1e6
ENV_RUN_TIME = 1000  # simulation run time
LC = 1e-24  # power usage constant
Y_NT = 0.5  # Weighted factor for local time cost
Y_NE = 0.5  # Weighted factor for energy cost

# Networking and Computational Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # task data in Mbit
max_power_consumption = 1000  # in Joules

# CPU and power parameters for devices and MEC server
local_device_cycle_freq = random.uniform(2, 4) * MHZ_TO_HZ  # in Hz
mec_server_cycle_freq = 4 * GHZ_TO_HZ  # MEC server capacity set to 4 GHz
local_device_power = random.uniform(100, 400) / 1000  # in Watts
clock_cycles_per_bit = random.randint(800, 1200)

# Channel parameters
B = 1e6  # Channel bandwidth in Hz
H0 = 1e-9  # Noise spectral density in W/Hz
P = 1e-3  # Communication capacity in W
alpha = 4  # Path loss exponent

class MECServer:
    def __init__(self, env, capacity):
        self.env = env
        self.capacity = capacity
        self.processor = simpy.Resource(env, capacity=int(capacity))
        self.dag = nx.DiGraph()

    def add_task_to_dag(self, task_id, dependencies, task_data):
        self.dag.add_node(task_id, data=task_data, vk=random.random())
        for dep in dependencies:
            self.dag.add_edge(dep, task_id)
        print(f"Added task {task_id} to DAG with dependencies {dependencies} and data {task_data}")

    def process_task(self, task_id):
        task_info = self.dag.nodes[task_id]
        data_bits = task_info['data'] * BIT_TO_MBIT * 1e6
        clock_cycles = data_bits * clock_cycles_per_bit

        edge_time = clock_cycles / self.capacity
        print(f"Processing task {task_id} with edge time {edge_time}")

        yield self.env.timeout(edge_time)
        self.dag.nodes[task_id]['ct'] = self.env.now

    def execute_tasks(self):
        for task in nx.topological_sort(self.dag):
            yield self.env.process(self.process_task(task))

class Device:
    def __init__(self, env, mec_servers, logs, num_devices):
        self.env = env
        self.mec_servers = mec_servers
        self.logs = logs
        self.num_devices = num_devices
        self.Q0 = []
        self.Q1 = []

    def local_computation_model(self, task_data):
        Hn = task_data * BIT_TO_MBIT * 1e6  # Convert Mbit to bits
        Rn = clock_cycles_per_bit  # CPU cycles per bit
        Qn = 0  # No offloading, process locally
        local_cycles = Hn * Rn * (1 - Qn)
        RTl = local_cycles / local_device_cycle_freq
        ATl = LC * local_device_cycle_freq**2 * local_cycles
        DTl = Y_NT * RTl + Y_NE * ATl
        print(f"Local computation model: RTl={RTl}, ATl={ATl}, DTl={DTl}")
        return RTl, ATl, DTl

    def mec_computation_model(self, task_data, Qn):
        mec = min(self.mec_servers, key=lambda x: len(x.dag))
        task_id = self.env.now
        mec.add_task_to_dag(task_id, [], task_data)

        # Calculate the local computation parameters
        Hn = task_data * BIT_TO_MBIT * 1e6  # Convert Mbit to bits
        Rn = clock_cycles_per_bit  # CPU cycles per bit

        # Calculate transmission rate and time
        RQx = local_device_power  # Assuming transmission power is the same as device power
        distance = random.uniform(10, 100)  # Distance between device and MEC server in meters
        DV_x_m = (distance**-alpha)  # Channel gain
        Wx = B * np.log2(1 + (RQx * DV_x_m) / (H0 * P))
        if Wx <= 0:
            Wx = 1e-9  # Avoid divide by zero
        transmission_time = (Hn * Qn) / Wx
        print(f"MEC computation model: transmission_time={transmission_time}, Wx={Wx}")

        # Calculate energy consumption for transmission
        transmission_energy = RQx * transmission_time

        # Process task on MEC server
        edge_time = Hn * Rn * Qn / mec.capacity
        yield self.env.timeout(transmission_time)
        yield self.env.process(mec.execute_tasks())
        if 'ct' in mec.dag.nodes[task_id]:
            ct = mec.dag.nodes[task_id]['ct']
        else:
            ct = self.env.now

        # Calculate edge computation parameters
        ATm = LC * mec.capacity**2 * (Hn * Rn * Qn)
        DTm = transmission_energy + ATm
        print(f"MEC computation model: edge_time={edge_time}, ATm={ATm}, DTm={DTm}")

        return transmission_time + edge_time, DTm, Wx

    def send_task(self, task_data, dependencies=[]):
        Qn = random.uniform(0, 1)  # Offloading decision
        if Qn == 0:  # Local computation
            RTl, ATl, DTl = self.local_computation_model(task_data)
            yield self.env.timeout(RTl)
            total_time = RTl
            energy_consumed = ATl
            trans_rate = 0
        else:  # MEC computation
            RTm, DTm, trans_rate = yield self.env.process(self.mec_computation_model(task_data, Qn))
            total_time = RTm
            energy_consumed = DTm
        print(f"Task {self.env.now}: total_time={total_time}, energy_consumed={energy_consumed}, Qn={Qn}, trans_rate={trans_rate}")

        self.logs.append({
            'num_devices': self.num_devices,
            'task_data': task_data,
            'completion_time': total_time,
            'energy_consumed': energy_consumed,
            'weighted_waste': DTl if Qn == 0 else DTm,
            'transmission_rate': trans_rate  # Log the actual transmission rate
        })

env = simpy.Environment()
mecs = [MECServer(env, mec_server_cycle_freq) for _ in range(10)]  # Number of MEC servers set to 10
logs = []

for num_devices in num_devices_list:
    devices = [Device(env, mecs, logs, num_devices) for _ in range(num_devices)]
    for device in devices:
        task_size = random.choice(task_data_sizes)
        env.process(device.send_task(task_size))

env.run(until=ENV_RUN_TIME)

df_logs = pd.DataFrame(logs)
print(df_logs.head())
if df_logs.empty:
    print("No logs were generated.")
else:
    if df_logs['energy_consumed'].max() > max_power_consumption:
        print("Warning: Power consumption exceeded the maximum limit.")


Added task 0 to DAG with dependencies [] and data 15
MEC computation model: transmission_time=0.0002559504201041921, Wx=11707066.159603754
Added task 0 to DAG with dependencies [] and data 35
MEC computation model: transmission_time=0.0016226566482900788, Wx=19025819.521863665
Added task 0 to DAG with dependencies [] and data 20
MEC computation model: transmission_time=0.0006328066023960096, Wx=14712748.976100346
Added task 0 to DAG with dependencies [] and data 15
MEC computation model: transmission_time=0.0008139813000984888, Wx=13285431.167535603
Added task 0 to DAG with dependencies [] and data 15
MEC computation model: transmission_time=0.0006123294946864263, Wx=16054423.331961641
Added task 0 to DAG with dependencies [] and data 5
MEC computation model: transmission_time=0.00011477139016284099, Wx=22156466.986073207
Added task 0 to DAG with dependencies [] and data 45
MEC computation model: transmission_time=0.001358599126033471, Wx=19033972.07980164
Added task 0 to DAG with depe

In [45]:
# Compute the metrics
mean_completion_by_devices = df_logs.groupby('num_devices')['completion_time'].sum()
mean_completion_by_task_data = df_logs.groupby('task_data')['completion_time'].mean()
mean_completion_by_transmission_rate = df_logs.groupby('transmission_rate')['completion_time'].mean()

energy_by_devices = df_logs.groupby('num_devices')['energy_consumed'].mean()
energy_by_task_data = df_logs.groupby('task_data')['energy_consumed'].sum()
energy_by_transmission_rate = df_logs.groupby('transmission_rate')['energy_consumed'].sum()

In [46]:
# Output the results
print("Mean Completion Times by Number of Devices:")
print(mean_completion_by_devices)

Mean Completion Times by Number of Devices:
num_devices
50     0.122342
100    0.361708
150    0.550638
200    0.599520
250    0.836582
300    0.904568
Name: completion_time, dtype: float64


In [41]:
print("\nMean Completion Times by Task Data:")
print(mean_completion_by_task_data)


Mean Completion Times by Task Data:
task_data
5     0.000675
10    0.001384
15    0.002062
20    0.002630
25    0.003251
30    0.003924
35    0.004383
40    0.005235
45    0.005388
Name: completion_time, dtype: float64


In [42]:
print("\nMean Completion Times by Transmission Rate:")
print(mean_completion_by_transmission_rate)


Mean Completion Times by Transmission Rate:
transmission_rate
1.191830e+07    0.005158
1.191834e+07    0.001434
1.191848e+07    0.003527
1.192256e+07    0.001189
1.192554e+07    0.004001
                  ...   
2.505706e+07    0.003817
2.514020e+07    0.005442
2.514428e+07    0.002591
2.517742e+07    0.004345
2.518024e+07    0.001270
Name: completion_time, Length: 1050, dtype: float64


In [43]:
print("\nTotal Energy Consumption by Number of Devices:")
print(energy_by_devices)


Total Energy Consumption by Number of Devices:
num_devices
50      5897.085399
100    17429.576879
150    26893.372695
200    29193.854406
250    40427.125903
300    44196.641025
Name: energy_consumed, dtype: float64


In [44]:
print("\nTotal Energy Consumption by Task Data:")
print(energy_by_task_data)


Total Energy Consumption by Task Data:
task_data
5      3893.934354
10     8255.786454
15    10821.949978
20    14343.833129
25    16944.767379
30    24544.647492
35    25520.080849
40    29661.661372
45    30050.995301
Name: energy_consumed, dtype: float64


In [ ]:
print("\nTotal Energy Consumption by Transmission Rate:")
print(energy_by_transmission_rate)


Total Energy Consumption by Transmission Rate:
transmission_rate
500.0     39077.189239
1000.0    39790.168873
1500.0    42611.550284
2000.0    29507.904894
2500.0    31778.786233
3000.0    37075.837815
Name: energy_consumed, dtype: float64
